# Classical-quantum trajectories

In this notebook we generate the example classical-quantum trajectories appearing in the Appendix of [arxiv XXXXX/XXXXXX](https://www.google.com).

### Basics of the code

Let's start by importing the usual dependencies:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

The core goal in what follows is to simulate how a classical and quantum system interact. Their evolution may be represented entirely by the classical point in phase space $Z_s$ and the quantum state $\rho_s$ (or $|\psi\rangle_s$ for pure states) at each point in time $s$, and so these will intially appear in the code as

In [3]:
N=10**6 #number of timesteps in simulation

Z_s=np.zeros(N + 1)
rho_s = np.zeros((N + 1,2,2),dtype = 'complex_')
psi_s = np.zeros((N + 1,2),dtype = 'complex_')

The remaining task is to fill in these blank vectors by a suitable numerical approximation to the dynamics. The dynamics generating these trajectories are generally stochastic differential equations of the form

$$ dZ_t=\mu(\rho,Z) dt + \sigma(Z) dW $$
$$ d\rho_t=f(\rho,Z) dt + g(\rho,Z) dW $$

As the applications of the current work are to illustrate how classical-quantum trajectories appear in toy models and provide proof-of-principle for their simulation, we will only consider the most basic of stochastic finite difference schemes, known as the [Euler-Maruyama method](https://en.wikipedia.org/wiki/Euler%E2%80%93Maruyama_method). The idea here is to generate the Wiener increment via

In [4]:
state=np.random.RandomState(seed=1) #fix a seed

def dW(delta_t):
    return state.normal(loc=0.0, scale=np.sqrt(delta_t))

and then simulate the stochastic differential equation by considering the above stochastic differential equations as defining the increment to add to $Z_t$ or $\rho_t$ at each timestep to find $Z_{t+\delta t}$,$\rho_{t+\delta t}$. Having done this, we may plot the relevant trajectory in phase space and Hilbert space using a suitable representation such as the Bloch sphere. Since it is not necessary for high numerical accuracy to understand the behaviour of these toy systems, we will assume that provided the trace of the quantum system is close to 1 the numerical model is behaving suitably well – in most cases this can be simply verified by looking at the path of the trajectory in Hilbert space.